<a href="https://colab.research.google.com/github/snehalgore1/DBMS-PROJECT/blob/main/Stock_Market_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime as dt
from datetime import date
import pandas as pd
import pandas_datareader.data as web
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
#Getting the historical data from yahoo finance

today = date.today()
print(today)
tdy = (today.strftime('%Y,%m,%d')).split(',')
print(tdy)
start = dt.datetime(2000,6,1)
end = dt.datetime(int(tdy[0]),int(tdy[1]),int(tdy[2]))
df = web.DataReader('MSFT' , 'yahoo' , start , end)
print(df)

2022-02-19
['2022', '02', '19']
                  High         Low  ...      Volume   Adj Close
Date                                ...                        
2000-06-01   33.000000   31.906250  ...  94507200.0   20.337797
2000-06-02   33.375000   32.500000  ...  80463200.0   20.889061
2000-06-05   34.187500   33.000000  ...  64400000.0   21.066259
2000-06-06   34.937500   33.906250  ...  99019000.0   21.932539
2000-06-07   35.375000   33.562500  ...  76310600.0   22.208168
...                ...         ...  ...         ...         ...
2022-02-14  296.760010  291.350006  ...  36339400.0  294.391296
2022-02-15  300.799988  297.019989  ...  27379500.0  299.850006
2022-02-16  300.869995  293.679993  ...  29982100.0  299.500000
2022-02-17  296.799988  290.000000  ...  32461600.0  290.730011
2022-02-18  293.859985  286.309998  ...  34223200.0  287.929993

[5466 rows x 6 columns]


In [ ]:
#Extracting Dates
l = list(df.index)

In [ ]:
#Converted into Dataframe
df = pd.DataFrame(df, columns = df.columns)
print (df.head)

<bound method NDFrame.head of                   High         Low  ...      Volume   Adj Close
Date                                ...                        
2000-06-01   33.000000   31.906250  ...  94507200.0   20.337797
2000-06-02   33.375000   32.500000  ...  80463200.0   20.889061
2000-06-05   34.187500   33.000000  ...  64400000.0   21.066259
2000-06-06   34.937500   33.906250  ...  99019000.0   21.932539
2000-06-07   35.375000   33.562500  ...  76310600.0   22.208168
...                ...         ...  ...         ...         ...
2022-02-14  296.760010  291.350006  ...  36339400.0  294.391296
2022-02-15  300.799988  297.019989  ...  27379500.0  299.850006
2022-02-16  300.869995  293.679993  ...  29982100.0  299.500000
2022-02-17  296.799988  290.000000  ...  32461600.0  290.730011
2022-02-18  293.859985  286.309998  ...  34223200.0  287.929993

[5466 rows x 6 columns]>


In [ ]:
#Adding the next 29 adj. close values
for i in range(1,30) :
  forecast_out = i
  df['Adj Close'+str(i)] = df[['Adj Close']].shift(-forecast_out)
df.head()

,High,Low,Open,Close,Volume,Adj Close,Adj Close1,Adj Close2,Adj Close3,Adj Close4,Adj Close5,Adj Close6,Adj Close7,Adj Close8,Adj Close9,Adj Close10,Adj Close11,Adj Close12,Adj Close13,Adj Close14,Adj Close15,Adj Close16,Adj Close17,Adj Close18,Adj Close19,Adj Close20,Adj Close21,Adj Close22,Adj Close23,Adj Close24,Adj Close25,Adj Close26,Adj Close27,Adj Close28,Adj Close29
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-06-01,33.0000,31.90625,32.187500,32.28125,94507200.0,20.337797,20.889061,21.066259,21.932539,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074
2000-06-02,33.3750,32.50000,33.000000,33.15625,80463200.0,20.889061,21.066259,21.932539,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074,24.866064
2000-06-05,34.1875,33.00000,33.007812,33.43750,64400000.0,21.066259,21.932539,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074,24.866064,24.629801
2000-06-06,34.9375,33.90625,34.093750,34.81250,99019000.0,21.932539,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074,24.866064,24.629801,24.728230
2000-06-07,35.3750,33.56250,34.625000,35.25000,76310600.0,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074,24.866064,24.629801,24.728230,23.035072


In [ ]:
# A variable for predicting 'n' days out into the future
forecast_out = 30 #'n=30' days
#Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Adj Close']].shift(-forecast_out)
#print the new data set
df.head()

,High,Low,Open,Close,Volume,Adj Close,Adj Close1,Adj Close2,Adj Close3,Adj Close4,Adj Close5,Adj Close6,Adj Close7,Adj Close8,Adj Close9,Adj Close10,Adj Close11,Adj Close12,Adj Close13,Adj Close14,Adj Close15,Adj Close16,Adj Close17,Adj Close18,Adj Close19,Adj Close20,Adj Close21,Adj Close22,Adj Close23,Adj Close24,Adj Close25,Adj Close26,Adj Close27,Adj Close28,Adj Close29,Prediction
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-06-01,33.0000,31.90625,32.187500,32.28125,94507200.0,20.337797,20.889061,21.066259,21.932539,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074,24.866064
2000-06-02,33.3750,32.50000,33.000000,33.15625,80463200.0,20.889061,21.066259,21.932539,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074,24.866064,24.629801
2000-06-05,34.1875,33.00000,33.007812,33.43750,64400000.0,21.066259,21.932539,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074,24.866064,24.629801,24.728230
2000-06-06,34.9375,33.90625,34.093750,34.81250,99019000.0,21.932539,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074,24.866064,24.629801,24.728230,23.035072
2000-06-07,35.3750,33.56250,34.625000,35.25000,76310600.0,22.208168,21.676584,21.676584,21.066259,21.381269,22.208168,22.798809,22.857874,23.212263,23.606022,25.417326,25.161385,24.472298,25.043255,24.826674,24.866064,24.314793,25.200752,25.200752,24.728230,25.496075,25.830770,25.023561,24.925129,25.299198,25.181074,24.866064,24.629801,24.728230,23.035072,23.566647


In [ ]:
# Create the independent data set (x)
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))
#Remove the last 'n' rows
X = X[: -forecast_out]
print(X)

[[ 33.          31.90625     32.1875     ...  24.92512894  25.29919815
   25.18107414]
 [ 33.375       32.5         33.         ...  25.29919815  25.18107414
   24.86606407]
 [ 34.1875      33.          33.0078125  ...  25.18107414  24.86606407
   24.6298008 ]
 ...
 [335.20001221 326.11999512 334.82998657 ... 294.43121338 294.39129639
  299.8500061 ]
 [326.07000732 315.98001099 325.85998535 ... 294.39129639 299.8500061
  299.5       ]
 [318.70001221 311.48999023 313.1499939  ... 299.8500061  299.5
  290.73001099]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
#Data Standardization
dfs = preprocessing.scale(X)

In [27]:
X=dfs
print(X)

[[-0.4293161  -0.43557105 -0.43649396 ... -0.44522576 -0.43997956
  -0.44194786]
 [-0.42353423 -0.4262473  -0.42385409 ... -0.43974879 -0.44170709
  -0.44654986]
 [-0.41100686 -0.41839573 -0.42373255 ... -0.44147831 -0.44631401
  -0.45000143]
 ...
 [ 4.23009492  4.18451219  4.27164534 ...  3.50077293  3.49540693
   3.57070054]
 [ 4.08932575  4.02528247  4.1321011  ...  3.50018848  3.57523883
   3.56558729]
 [ 3.97569294  3.95477499  3.9343747  ...  3.58011269  3.5701201
   3.4374662 ]]


In [29]:
##Create the dependent data set (y) ##
# Convert the dataframe to a numpy array
y = np.array(df['Prediction'])
#Get all of the y values except the last 'n' rows
y = y[:-forecast_out]
print(y)

[ 24.86606407  24.6298008   24.72822952 ... 299.5        290.73001099
 287.92999268]


In [30]:
#Split the data into 60% training and 40% temp

x_train, x_temp, y_train, y_temp = train_test_split(X,y,test_size =0.4,shuffle=False)

In [33]:
#Split the temp data half-half into test and forecast (20% test 20% forecast)

x_test, x_forecast, y_test, y_forecast = train_test_split(x_temp,y_temp,test_size =0.5,shuffle=False)

In [42]:
# Create and train the Linear Regression Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression()

In [54]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction.
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test) 
print("Linear Regression confidence: ", lr_confidence)

Linear Regression confidence:  0.997078159383159


In [ ]:
#Training Random Forest Model
rf = RandomForestRegressor(n_estimators=500,random_state=0)
rf.fit(x_train,y_train)

RandomForestRegressor(n_estimators=500, random_state=0)

In [65]:
rf_confidence = rf.score(x_test,y_test)
print("Random Forest Confidence: ",rf_confidence)

Random Forest Confidence:  -1.720722227502522
